## *Entity Extraction from descripiton related to a book using Granite-8B*
LLMs have demonstrated remarkable accuracy in the task of entity extraction. This cookbook focuses on extracting key entities from descriptions related to books

### Install dependencies

In [1]:
!pip install git+https://github.com/ibm-granite-community/utils langchain_community pydantic

  Cloning https://github.com/ibm-granite-community/utils to /private/var/folders/yq/mg65c_l16hv64plnb99z5dx40000gq/T/pip-req-build-4kuap8v3
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /private/var/folders/yq/mg65c_l16hv64plnb99z5dx40000gq/T/pip-req-build-4kuap8v3
  Resolved https://github.com/ibm-granite-community/utils to commit 5d67648927240b208a164d2466f0dc77200450e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ibm-granite-community-utils: filename=ibm_granite_community_utils-0.1.dev49-py3-none-any.whl size=8536 sha256=e10e9e7c4bf8e4fe0448b2b751d8640c3717efe2066b186f29232d06ebd97abc
  Stored in directory: /private/var/folders/yq/mg65c_l16hv64plnb99z5dx40000gq/T/pip-ephem-wheel-cache-mtemf8hf/wheels/ee/16/b7/e10e3986779ca7e437370ddd3854cbbb6f9ddc27acf97a1405
Successfully built ibm-granite-community-utils
  Attempting u

### Instantiate the model client

In [1]:
import json
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from ibm_granite_community.notebook_utils import get_env_var

model_name: str = "granite3.1-dense:8b"

model =  OllamaLLM(
        model=model_name,
        temperature=0
    )

### 1 - Entity Extraction by defining entities in the prompt

The first approach is straightforward and involves explicitly defining the entities within the prompt itself. In this method, we specify the entities to be extracted along with their descriptions directly in the prompt. This includes:  

<u>**Entity Definitions:**</u> Each entity, such as title, author, price, and rating, is clearly outlined with a concise description of what it represents.  

<u>**Prompt Structure:**</u> The prompt is structured to guide the LLM in understanding exactly what information is needed. By providing detailed instructions, we aim to ensure that the model focuses on extracting only the relevant data.  

<u>**Output Format:**</u> The output is required to be in JSON format, which enforces a consistent structure for the extracted data. If any entity is not found, the model is instructed to return "Data not available," preventing ambiguity.  

Provide some text with information for a book. In this case, we use generated commentary on 'The Hunger Games' by Suzanne Collins.

In [2]:
book_info = """Notice of Representation

Budget Mutual Insurance Company 9876 Infinity Ave Springfield, MI 65541

Georgia Collan Parker LLP 9816 51st Ave SW Auburn, Washington(WA), 98092

Our Client: Courtney Sosa Date of death: 6/12/2020

To Whom It May Concern,

I have been retained by Courtney Sosa to handle the estate of Lukas Juarez. My understanding is that they had a life insurance policy (#951033310) with your company. If this is correct, please send a letter to my office indicating you have received our letter of representation. Additionally, please do not contact our client going forward.

We are requesting that you forward the full policy amount of $50,000. Please forward an acknowledgement of our demand and please forward the umbrella policy information if one is applicable. Please send my secretary any information regarding liens on his policy.

Please contact my office if you have any questions.

Sincerely,

Angela Berry, Attorney
"""

All the entities that need to be fetched are defined in the prompt itself along with the entity's description.

In [3]:
entity_prompt = f"""
<|start_of_role|>user<|end_of_role|>
    -You are an AI Entity Extractor. You help extract entities from the given information about a book. Here is the book information:
    {book_info}

    - Extract the following entities:

    1) `Insurance Company` : This is the name of the company.
    2) `Insurance Company Address`: This is the address of the company.
    3) `Law Firm`: Name of the Law Firm.
    4) `Law Office Address`: This is the address of the law firm.

    -Your output should strictly be in a json format, which only contains the key and value. The key here is the entity to be extracted and the value is the entity which you extracted.
    -Do not generate random entities on your own. If it is not present or you are unable to find any specified entity, you strictly have to output it as `Data not available`.
    -Only do what is asked to you. Do not give any explanations to your output and do not hallucinate.
    <|end_of_text|>
    <|start_of_role|>assistant<|end_of_role|>
"""

Invoking the model to get the results

In [4]:
response = model.invoke(entity_prompt)
print(response)

{
  "Insurance Company": "Budget Mutual Insurance Company",
  "Insurance Company Address": "9876 Infinity Ave Springfield, MI 65541",
  "Law Firm": "Georgia Collan Parker LLP",
  "Law Office Address": "9816 51st Ave SW Auburn, Washington(WA), 98092"
}


In [5]:
book_info = json.loads(response)
book_info

{'Insurance Company': 'Budget Mutual Insurance Company',
 'Insurance Company Address': '9876 Infinity Ave Springfield, MI 65541',
 'Law Firm': 'Georgia Collan Parker LLP',
 'Law Office Address': '9816 51st Ave SW Auburn, Washington(WA), 98092'}

---